In [ ]:
import numpy as np
import pandas as pd
import scipy.stats

pd.options.mode.chained_assignment = None
from matplotlib import pyplot as plt, rcParams
# import cv2
import seaborn as sns

sns.set(style="white", context="paper")
from cycler import cycler
import os, sys
import glob
from datetime import datetime, timedelta
from itertools import combinations
import base64
from PIL import Image
from io import BytesIO as _BytesIO
import requests
import json
import pickle
from datetime import datetime
from IPython.display import display, Markdown, Latex
from sklearn.metrics import *
import collections
from copy import deepcopy
import traceback
from sympy import Point, Polygon
import mmcv
# import mmcv


import cv2
from mmtrack.apis import inference_mot, init_model as init_tracking_model
from smartprint import smartprint as sprint

pd.options.display.max_columns = None
def printm(s): return display(Markdown(s))
    
SERVER_CACHE_DIR = '/mnt/ci-nas-cache/edulyzeV2/cache_compute_4/fixed_face'
os.makedirs(SERVER_CACHE_DIR,exist_ok=True)

track_analysis_meta_cache = f'{SERVER_CACHE_DIR}/analysis_tracking/meta_info'
os.makedirs(track_analysis_meta_cache,exist_ok=True)

base_dir = '/mnt/ci-nas-cache/edulyzeV2/pose_face_gaze_emb_fixed_face'

track_analysis_session_data = f'{SERVER_CACHE_DIR}/analysis_tracking/session_tracking_info'
os.makedirs(track_analysis_session_data,exist_ok=True)

postprocessed_id_map_data_dir = f'{SERVER_CACHE_DIR}/analysis_tracking/processed_id_maps'
os.makedirs(postprocessed_id_map_data_dir, exist_ok=True)

id_viz_cache_root = f'{SERVER_CACHE_DIR}/analysis_tracking/session_matching_info'
os.makedirs(id_viz_cache_root, exist_ok=True)


In [ ]:
session_filter_list = [
#  'classinsight-cmu_05681A_ghc_4301_201905011630',
#  'classinsight-cmu_05681A_ghc_4301_201904171630',
#  'classinsight-cmu_05681A_ghc_4301_201902201630',
#  'classinsight-cmu_05681A_ghc_4301_201904101630',
#  'classinsight-cmu_05681A_ghc_4301_201901231630',
 # 'classinsight-cmu_05418A_ghc_4102_201902251200',
 # 'classinsight-cmu_05418A_ghc_4102_201904081200',
 # 'classinsight-cmu_05418A_ghc_4102_201905011200',
 # 'classinsight-cmu_05418A_ghc_4102_201904291200',
 # 'classinsight-cmu_05418A_ghc_4102_201904011200',
 # 'classinsight-cmu_05748A_ghc_4101_201902141630',
 # 'classinsight-cmu_05748A_ghc_4101_201904021630',
 # 'classinsight-cmu_05748A_ghc_4101_201902051630',
 # 'classinsight-cmu_05748A_ghc_4101_201902281630',
 # 'classinsight-cmu_05748A_ghc_4101_201903071630',
 # 'classinsight-cmu_21127J_ghc_4102_201904230930',
 # 'classinsight-cmu_21127J_ghc_4102_201903260930',
 # 'classinsight-cmu_21127J_ghc_4102_201904160930',
 # 'classinsight-cmu_21127J_ghc_4102_201904300930',
 # 'classinsight-cmu_21127J_ghc_4102_201903190930',
 # 'classinsight-cmu_05410A_ghc_4301_201904151500',
 # 'classinsight-cmu_05410A_ghc_4301_201902251500',
 # 'classinsight-cmu_05410A_ghc_4301_201904081500',
 # 'classinsight-cmu_05410A_ghc_4301_201904221500',
 # 'classinsight-cmu_05410A_ghc_4301_201902181500',
                       
 # 'classinsight-cmu_17214B_ph_a21_201902271030',
 # 'classinsight-cmu_17214B_ph_a21_201903061030',
 # 'classinsight-cmu_17214B_ph_a21_201904031030',
 # 'classinsight-cmu_17214B_ph_a21_201904101030',
 # 'classinsight-cmu_17214B_ph_a21_201904241030',
 # 'classinsight-cmu_17214C_ph_225b_201903201130',
 # 'classinsight-cmu_17214C_ph_225b_201904101130',
 # 'classinsight-cmu_17214C_ph_225b_201904171130',
 # 'classinsight-cmu_17214C_ph_225b_201904241130',
 # 'classinsight-cmu_17214C_ph_225b_201905011130',
 # 'classinsight-cmu_05410B_ghc_4211_201902111500',
 # 'classinsight-cmu_05410B_ghc_4211_201903181500',
 # 'classinsight-cmu_05410B_ghc_4211_201904081500',
 # 'classinsight-cmu_05410B_ghc_4211_201904151500',
 # 'classinsight-cmu_05410B_ghc_4211_201904221500',
 # 'classinsight-cmu_05410B_ghc_4211_201901281500'
]


# Cache and get tracking information for sessions


## Get frame file data for all sessions


In [ ]:

frame_file_data = {}
for course_idx, course_dir in enumerate(glob.glob(f"{base_dir}/*")):
    course_name = course_dir.split("/")[-1]
    course_cache_file = f"{track_analysis_meta_cache}/{course_name}"
    if os.path.exists(course_cache_file):
        frame_file_data[course_name] = pickle.load(open(course_cache_file,"rb"))
        continue
    frame_file_data[course_name]={}
        
    for session_idx, session_dir in enumerate(glob.glob(f"{course_dir}/*")):
        session_name = session_dir.split("/")[-1]
        frame_file_data[course_name][session_name] = {}
        frame_files = glob.glob(f"{session_dir}/*")
        frame_file_names = [xr.split("/")[-1] for xr in frame_files]
        if 'end.pb' in frame_file_names:
            frame_file_data[course_name][session_name]['is_completed']=True
        else:
            frame_file_data[course_name][session_name]['is_completed']=False            
        frame_ids = [int(xr.split(".")[0]) for xr in frame_file_names if not (xr=='end.pb')]
        frame_file_data[course_name][session_name]['frame_ids'] = sorted(frame_ids)
        frame_file_data[course_name][session_name]['dir_location'] = session_dir
        print(f"Got metadata for course: {course_idx}-{course_name}, session:{session_idx}-{session_name}")
    pickle.dump(frame_file_data[course_name],open(course_cache_file,"wb")) 
        
        
frame_file_data.keys()


## Collect tracking data for all sessions


In [ ]:
# writing a generic loop to get only-tracking info from all courses in frame file data, but putting a filter ahead

for course_idx, course in enumerate(frame_file_data):
    for session_idx, session_id in enumerate(frame_file_data[course]):
        session_tracking_cache_file = f"{track_analysis_session_data}/{session_id}.pb"
        # if session_id.split("-front")[0] not in session_filter_list:
        #     print(f"Session {session_id} not in session filter list, skipping...")
        #     continue
        try:
a                print(f"Got tracking info for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
            else:
                ...
                print(f"FILE EXISTS: tracking info for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
        except:
            print(f"ERROR: Unable to get session tracking for: {course_idx}-{course}, session:{session_idx}-{session_id}")
            unfinished_sessions.append((course, session_id))
            print(traceback.format_exc())
    

## Postprocess mmtrack results and create a new tracking map from older tracking ids


### Run id filtering and mapping for all sessions


In [ ]:

for course_idx, course in enumerate(frame_file_data):
    for session_idx, session_id in enumerate(frame_file_data[course]):
        session_preprocessed_id_map_file = f"{postprocessed_id_map_data_dir}/{session_id}.pb"
        if not os.path.exists(session_preprocessed_id_map_file):
            session_tracking_cache_file = f"{track_analysis_session_data}/{session_id}.pb"
            session_frame_dir = f'/mnt/ci-nas-cache/edulyzeV2/pose_face_gaze_emb_fixed_face/{course}/{session_id}'
            if not os.path.exists(session_tracking_cache_file):
                printm(f"## Tracking file does not exists for session {session_id}, skipping id-matching...")
                continue
            
            df_tracking = pickle.load(open(session_tracking_cache_file,"rb")).transpose()
            printm(f'# {course_idx}-{course}, session:{session_idx}-{session_id}')
            printm(f'## Raw tracking shape:{df_tracking.shape}')
            
            
            # Find other IDs that start/end within 900 frames
            printm(f'## Filter non-persistentids')
            MIN_ID_FRAMES = 900 # number of frames an id needs to be a persistent id
            col_start_stop_idxs = []
            for col in df_tracking.columns:
                one_idxs = df_tracking.index[np.where(df_tracking[col]==1)[0]].values
                col_start_stop_idxs.append([col, one_idxs.min(), one_idxs.max()])
            df_id_start_stop = pd.DataFrame(col_start_stop_idxs, columns=['id','min_idx','max_idx'])
            df_id_start_stop['total_idxs'] = df_id_start_stop['max_idx']-df_id_start_stop['min_idx']
            nonpersistent_ids_removed = df_id_start_stop[df_id_start_stop.total_idxs<=MIN_ID_FRAMES]['id'].values
            printm(f'### Total ids before filtering: {df_id_start_stop.shape[0]}')
            df_id_start_stop = df_id_start_stop[df_id_start_stop.total_idxs>MIN_ID_FRAMES].reset_index(drop=True)
            printm(f'### Total ids after filtering: {df_id_start_stop.shape[0]}')
            



            printm(f'## Map ids into one based on bbox overlap and id start/stop distance')
            MAX_ID_DISTANCE = 900
            MAX_BBOX_OVERLAP = 0.4
            
            potential_id_maps = {}
            num_possible_maps = 0 
            for row_idx, row in df_id_start_stop.iterrows():
                row_maxidx = row['max_idx']
                
                # get polygon for given id
                id_max_frame = row_maxidx
                id_frame_data = pickle.load(open(f"{session_frame_dir}/{id_max_frame}.pb","rb"))[1]
                id_frame_data = [xr for xr in id_frame_data if (xr['track_id']==row['id'])][0]
                id_bb = id_frame_data['bbox'][:4].astype(int)
                X_TL1, Y_TL1, X_BR1, Y_BR1 = id_bb
                p1, p2, p3, p4  = map(Point, [[X_TL1,Y_TL1], [X_TL1,Y_BR1], [X_BR1,Y_BR1],[X_BR1, Y_TL1]]) 
                id_polygon = Polygon(p1, p2, p3, p4)
                
                potential_id_matches = df_id_start_stop[(df_id_start_stop.min_idx<=row_maxidx+MAX_ID_DISTANCE) & (df_id_start_stop.min_idx>row_maxidx-MAX_ID_DISTANCE)].id.values
                successful_matches = []
                
                if len(potential_id_matches)>0:
                    num_possible_maps+=1
                    # print('\n',row['id'], potential_id_matches, row['min_idx'],row['max_idx'],row['total_idxs'])
                    for matched_id in potential_id_matches:
                        matched_id_min_frame = df_id_start_stop[df_id_start_stop.id==matched_id].min_idx.values[0]
                        matched_id_frame_data = pickle.load(open(f"{session_frame_dir}/{matched_id_min_frame}.pb","rb"))[1]
                        matched_id_frame_data = [xr for xr in matched_id_frame_data if (xr['track_id']==matched_id)][0]
                        matched_id_bb = matched_id_frame_data['bbox'][:4].astype(int)
                        X_TL2, Y_TL2, X_BR2, Y_BR2 = matched_id_bb
                        
                        p1, p2, p3, p4  = map(Point, [[X_TL2,Y_TL2], [X_TL2,Y_BR2], [X_BR2,Y_BR2],[X_BR2, Y_TL2]]) 
                        matched_id_polygon = Polygon(p1, p2, p3, p4)
                        
                        #find intersection of two polygons
                        # check if intersection exists
                        if id_polygon.encloses_point(matched_id_polygon.centroid) & matched_id_polygon.encloses_point(id_polygon.centroid):
                            X_TL_in, X_BR_in = sorted([X_TL1,X_TL2, X_BR1, X_BR2])[1:3]
                            Y_TL_in, Y_BR_in = sorted([Y_TL1,Y_TL2, Y_BR1, Y_BR2])[1:3]
                            p1, p2, p3, p4  = map(Point, [[X_TL_in,Y_TL_in], [X_TL_in,Y_BR_in], [X_BR_in,Y_BR_in],[X_BR_in, Y_TL_in]]) 
                            intersection = Polygon(p1, p2, p3, p4)            
            
                            #find polygon overlap
                            area_intersection = np.abs(intersection.area)
                            area_union = np.abs(id_polygon.area) + np.abs(matched_id_polygon.area) - area_intersection
                            overlap_fraction  = (area_intersection/area_union).evalf()
                        else:
                            overlap_fraction=0.            
                        if overlap_fraction > MAX_BBOX_OVERLAP:
                            successful_matches.append((matched_id, overlap_fraction))
                        
                        # print('\tMatching Id: ', matched_id,':', 'frame:',matched_id_min_frame,'overlap_fraction:', overlap_fraction)
                if len(successful_matches) > 0:
                    successful_matched_id = sorted(successful_matches, key=lambda x: x[1])[-1][0]
                    # print(row['id'], '-->Successful match to-->',successful_matched_id)
                    if row['id'] in potential_id_maps.keys():
                        potential_id_maps[successful_matched_id] = potential_id_maps[row['id']]
                    else:
                        potential_id_maps[successful_matched_id] = row['id']
            
            matched_ids  = list(potential_id_maps.keys())
            df_id_start_stop =  df_id_start_stop[~df_id_start_stop['id'].isin(matched_ids)].sort_values(by='id').reset_index(drop=True)
            printm(f'### Total ids after mapping: {df_id_start_stop.shape[0]}')
            
            printm(f'## Assign new ids to final set of postprocessed ids')
            new_to_old_id_map = df_id_start_stop['id'].to_dict()
            old_to_new_id_map = {v: k for k, v in new_to_old_id_map.items()}
            
            for matched_id in matched_ids:
                old_to_new_id_map[matched_id] = old_to_new_id_map[potential_id_maps[matched_id]]
            
            for removed_id in nonpersistent_ids_removed:
                old_to_new_id_map[removed_id] = 10000
            
            pickle.dump(old_to_new_id_map, open(session_preprocessed_id_map_file,"wb"))
            printm(f"## Created ID Map for session: {course_idx}-{course}, session:{session_idx}-{session_id} from {len(old_to_new_id_map.keys())} to {df_id_start_stop.shape[0]} ids")
        else:
            printm(f"## FILE EXISTS: ID Map for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
            
        
        

# Create Visualization from each session to collect id ground truth


In [ ]:
sample_course = '05748A'
sample_session_id = 'classinsight-cmu_05748A_ghc_4101_201902141630-front'
session_tracking_cache_file = f"{track_analysis_session_data}/{sample_session_id}.pb"
session_preprocessed_id_map_file = f"{postprocessed_id_map_data_dir}/{sample_session_id}.pb"
session_frame_dir = f'{base_dir}/{sample_course}/{sample_session_id}'
session_video_file = f'/mnt/ci-nas-classes/classinsight/2019S/video_backup/{sample_session_id.split("-front")[0]}/{sample_session_id}.avi'
session_frame_dir


In [ ]:
df_tracking_new = pickle.load(open(session_tracking_cache_file,"rb")).transpose()
old_to_new_id_map = pickle.load(open(session_preprocessed_id_map_file,"rb"))
total_idxs = df_tracking_new.index.max()
for old_id in old_to_new_id_map:
    new_id = old_to_new_id_map[old_id]
    if not new_id==10000:
        new_id_col = f'N{new_id}'
        if new_id_col not in df_tracking_new:
            df_tracking_new[new_id_col] = None
        df_tracking_new[new_id_col] =  df_tracking_new[old_id].where(~df_tracking_new[old_id].isnull(), df_tracking_new[old_id])
    df_tracking_new = df_tracking_new.drop(old_id, axis=1)


In [ ]:
df_tracking_new.shape


In [ ]:
total_idxs


In [ ]:
# old_to_new_id_map


In [ ]:
col_start_stop_idxs = []
for col in df_tracking_new.columns:
    one_idxs = df_tracking_new.index[np.where(df_tracking_new[col]==1)[0]].values
    col_start_stop_idxs.append([col, one_idxs.min(), one_idxs.max()])
df_id_start_stop = pd.DataFrame(col_start_stop_idxs, columns=['id','min_idx','max_idx'])
df_id_start_stop['total_idxs'] = df_id_start_stop['max_idx']-df_id_start_stop['min_idx']
df_id_start_stop['id'] = df_id_start_stop['id'].apply(lambda x: int(x[1:]))
df_id_start_stop


In [ ]:
_ = plt.figure(figsize=(20,15))
for row_idx, row in df_id_start_stop.iterrows():
    plt.axhline(y=row_idx, xmin=row['min_idx']/total_idxs,xmax=row['max_idx']/total_idxs)
plt.yticks(range(df_id_start_stop.shape[0]), range(df_id_start_stop.shape[0]))
plt.grid() 


In [ ]:
session_frames = df_tracking_new.index.values.tolist()
session_ids_covered = []
selected_frames = []
segments = []
segment_half_size = 50
for row_idx, row in df_id_start_stop.sort_values(by='total_idxs').iterrows():
    id_start, id_stop = row['min_idx'],row['max_idx']
    seg_mid = (id_start+id_stop)/2
    seg_start, seg_end = max(seg_mid-segment_half_size, 0), min(seg_mid+segment_half_size, total_idxs)
    is_id_covered = df_id_start_stop.apply(lambda row: (seg_end<=row['max_idx']) & (seg_start>=row['min_idx']),axis=1)
    seg_ids_covered = df_id_start_stop[is_id_covered]['id']
    new_ids_covered = [xr for xr in seg_ids_covered if xr not in session_ids_covered]
    if len(new_ids_covered)>0:        
        segment_frames = [xr for xr in session_frames if ((xr>=seg_start) and (xr<=seg_end))]
        selected_frames+=segment_frames
        segments.append((seg_start, seg_end, new_ids_covered, segment_frames))
        session_ids_covered+=new_ids_covered

sprint(segments)
    
    


In [ ]:
# for given frame ranges get video frames and tracking results
all_frames = mmcv.VideoReader(session_video_file)

required_frame_ids = np.unique(sorted(selected_frames))
frame_data_dict = {}

for frame_idx, frame_img in enumerate(all_frames):
    if (frame_idx in required_frame_ids) & (frame_idx%3==0):
        frame_data_dict[frame_idx] = frame_img
    if frame_idx%10000==0:
        print(f"Looped {frame_idx} images")
        



In [ ]:
SOURCE_ROOT = '/home/prasoon/video_analysis/edusenseV2compute/compute/videoV3'
run_config = {
    'track_config':f'{SOURCE_ROOT}/configs/mmlab/ocsort_yolox_x_crowdhuman_mot17-private-half.py',
    'track_checkpoint':f'{SOURCE_ROOT}/models/mmlab/ocsort_yolox_x_crowdhuman_mot17-private-half_20220813_101618-fe150582.pth',
    'device':'cuda:1',
}


In [ ]:
len(frame_data_dict.keys())


In [ ]:
session_id_viz_dir = f'{id_viz_cache_root}/{sample_session_id}'
os.makedirs(session_id_viz_dir, exist_ok=True)


In [ ]:
if tracking_model:
    printm("# Deleting Previous Model")
    del tracking_model
tracking_model = init_tracking_model(run_config['track_config'],
                                 run_config['track_checkpoint'],
                                 device=run_config['device'])
for seg_idx, (seg_start, seg_end, new_ids_covered, segment_frames) in enumerate(segments):
    seg_video_id = f'S{seg_idx}_{int(seg_start)}_{int(seg_end)}_{"_".join(map(str,new_ids_covered))}'
    match_cache_dir = f'{session_id_viz_dir}/{seg_video_id}'
    match_video_file = f'{session_id_viz_dir}/{seg_video_id}.mp4'
    if os.path.exists(match_video_file):
        continue
    print(match_video_file)
    os.makedirs(match_cache_dir, exist_ok=True)
    frame_num=0

    for frame_idx in segment_frames:
        frame_tracking_file = f"{session_frame_dir}/{frame_idx}.pb"
        if os.path.exists(frame_tracking_file) and (frame_idx in frame_data_dict):
            frame_file_out = os.path.join(match_cache_dir, f'{frame_num:06d}.jpg')
            frame_num+=1
            if os.path.exists(frame_file_out):
                continue
            frame_tracking_results = pickle.load(open(frame_tracking_file,"rb"))[1]
            for person_idx in range(len(frame_tracking_results)):
                old_id= int(frame_tracking_results[person_idx]['track_id'])
                frame_tracking_results[person_idx]['track_id'] =old_to_new_id_map[old_id]
            frame_img = deepcopy(frame_data_dict[frame_idx])
            if len(frame_tracking_results)>0:
                frame_tracking_results = {
                    'track_bboxes':[np.array([[xr['track_id']]+xr['bbox'].tolist() for xr in frame_tracking_results])],
                    'det_bboxes':[np.array([xr['bbox'].tolist() for xr in frame_tracking_results])]}
                frame_track_img = tracking_model.show_result(
                    frame_img,
                    frame_tracking_results,
                    thickness=5,
                    font_scale=.5,
                    score_thr=0.1,
                    show=False,
                    wait_time=int(1000. / 5),
                    out_file=frame_file_out,
                    backend='cv2')
            else:
                cv2.imwrite(frame_file_out,frame_img)
    
    #         break
    # break
    mmcv.frames2video(match_cache_dir, match_video_file, fps=5, fourcc='mp4v')
    print(f"Done for segment {seg_video_id}")
    # break
    
    

In [ ]:
id_viz_cache_root


# Run visualization code on all sessions


In [ ]:
SOURCE_ROOT = '/home/prasoon/video_analysis/edusenseV2compute/compute/videoV3'
run_config = {
    'track_config':f'{SOURCE_ROOT}/configs/mmlab/ocsort_yolox_x_crowdhuman_mot17-private-half.py',
    'track_checkpoint':f'{SOURCE_ROOT}/models/mmlab/ocsort_yolox_x_crowdhuman_mot17-private-half_20220813_101618-fe150582.pth',
    'device':'cuda:1',
}

for course_idx, course in enumerate(frame_file_data):
    for session_idx, session_id in enumerate(frame_file_data[course]):
        session_id_viz_dir = f'{id_viz_cache_root}/{session_id}'
        if not os.path.exists(session_id_viz_dir):
            
            session_tracking_cache_file = f"{track_analysis_session_data}/{session_id}.pb"
            session_preprocessed_id_map_file = f"{postprocessed_id_map_data_dir}/{session_id}.pb"
            session_frame_dir = f'/mnt/ci-nas-cache/edulyzeV2/pose_face_gaze_emb/{course}/{session_id}'
            session_video_file = f'/mnt/ci-nas-classes/classinsight/2019S/video_backup/{session_id.split("-front")[0]}/{session_id}.avi'
            printm(f'# {course_idx}-{course}, session:{session_idx}-{session_id}')
            printm("## Get preprocessed tracking ids...")
            
            df_tracking_new = pickle.load(open(session_tracking_cache_file,"rb")).transpose()
            old_to_new_id_map = pickle.load(open(session_preprocessed_id_map_file,"rb"))
            total_idxs = df_tracking_new.index.max()
            for old_id in old_to_new_id_map:
                new_id = old_to_new_id_map[old_id]
                if not new_id==10000:
                    new_id_col = f'N{new_id}'
                    if new_id_col not in df_tracking_new:
                        df_tracking_new[new_id_col] = None
                    df_tracking_new[new_id_col] =  df_tracking_new[old_id].where(~df_tracking_new[old_id].isnull(), df_tracking_new[old_id])
                df_tracking_new = df_tracking_new.drop(old_id, axis=1)

            col_start_stop_idxs = []
            for col in df_tracking_new.columns:
                one_idxs = df_tracking_new.index[np.where(df_tracking_new[col]==1)[0]].values
                col_start_stop_idxs.append([col, one_idxs.min(), one_idxs.max()])
            df_id_start_stop = pd.DataFrame(col_start_stop_idxs, columns=['id','min_idx','max_idx'])
            df_id_start_stop['total_idxs'] = df_id_start_stop['max_idx']-df_id_start_stop['min_idx']
            df_id_start_stop['id'] = df_id_start_stop['id'].apply(lambda x: int(x[1:]))
            
            printm(f"## Get segments to visualize from {df_tracking_new.shape[1]} tracking ids...")
            session_frames = df_tracking_new.index.values.tolist()
            session_ids_covered = []
            selected_frames = []
            segments = []
            segment_half_size = 50
            for row_idx, row in df_id_start_stop.sort_values(by='total_idxs').iterrows():
                id_start, id_stop = row['min_idx'],row['max_idx']
                seg_mid = (id_start+id_stop)/2
                seg_start, seg_end = max(seg_mid-segment_half_size, 0), min(seg_mid+segment_half_size, total_idxs)
                is_id_covered = df_id_start_stop.apply(lambda row: (seg_end<=row['max_idx']) & (seg_start>=row['min_idx']),axis=1)
                seg_ids_covered = df_id_start_stop[is_id_covered]['id']
                new_ids_covered = [xr for xr in seg_ids_covered if xr not in session_ids_covered]
                if len(new_ids_covered)>0:        
                    segment_frames = [xr for xr in session_frames if ((xr>=seg_start) and (xr<=seg_end))]
                    selected_frames+=segment_frames
                    segments.append((seg_start, seg_end, new_ids_covered, segment_frames))
                    session_ids_covered+=new_ids_covered
            printm(f"## Got {len(segments)} segments to visualize...")

            printm(f"## Get video frames from session video file for segment frames...")
            all_frames = mmcv.VideoReader(session_video_file)
            required_frame_ids = np.unique(sorted(selected_frames))
            frame_data_dict = dict()
            
            for frame_idx, frame_img in enumerate(all_frames):
                if (frame_idx in required_frame_ids) & (frame_idx%3==0):
                    frame_data_dict[frame_idx] = frame_img
                if frame_idx%1000==0:
                    print(f"Looped {frame_idx} images")
            printm(f"## Got {len(frame_data_dict.keys())} video frames from session video file...")

            printm(f"## Create segment videos with new tracking ids...")
            os.makedirs(session_id_viz_dir, exist_ok=True)
            tracking_model = init_tracking_model(run_config['track_config'],
                                             run_config['track_checkpoint'],
                                             device=run_config['device'])
            for seg_idx, (seg_start, seg_end, new_ids_covered, segment_frames) in enumerate(segments):
                seg_video_id = f'S{seg_idx}_{int(seg_start)}_{int(seg_end)}_{"_".join(map(str,new_ids_covered))}'
                match_cache_dir = f'{session_id_viz_dir}/{seg_video_id}'
                match_video_file = f'{session_id_viz_dir}/{seg_video_id}.mp4'
                if os.path.exists(match_video_file):
                    continue
                print(match_video_file)
                os.makedirs(match_cache_dir, exist_ok=True)
                frame_num=0
            
                for frame_idx in segment_frames:
                    frame_tracking_file = f"{session_frame_dir}/{frame_idx}.pb"
                    if os.path.exists(frame_tracking_file) and (frame_idx in frame_data_dict):
                        frame_file_out = os.path.join(match_cache_dir, f'{frame_num:06d}.jpg')
                        frame_num+=1
                        if os.path.exists(frame_file_out):
                            continue
                        frame_tracking_results = pickle.load(open(frame_tracking_file,"rb"))[1]
                        for person_idx in range(len(frame_tracking_results)):
                            old_id= int(frame_tracking_results[person_idx]['track_id'])
                            frame_tracking_results[person_idx]['track_id'] =old_to_new_id_map[old_id]
                        frame_img = deepcopy(frame_data_dict[frame_idx])
                        if len(frame_tracking_results)>0:
                            frame_tracking_results = {
                                'track_bboxes':[np.array([[xr['track_id']]+xr['bbox'].tolist() for xr in frame_tracking_results])],
                                'det_bboxes':[np.array([xr['bbox'].tolist() for xr in frame_tracking_results])]}
                            frame_track_img = tracking_model.show_result(
                                frame_img,
                                frame_tracking_results,
                                thickness=5,
                                font_scale=.5,
                                score_thr=0.1,
                                show=False,
                                wait_time=int(1000. / 5),
                                out_file=frame_file_out,
                                backend='cv2')
                        else:
                            cv2.imwrite(frame_file_out,frame_img)
                mmcv.frames2video(match_cache_dir, match_video_file, fps=5, fourcc='mp4v')
                print(f"Done for segment {seg_video_id}")
            del tracking_model
            printm(f"## Created Segment Visualization for session: {course_idx}-{course}, session:{session_idx}-{session_id} from {len(old_to_new_id_map.keys())} to {df_id_start_stop.shape[0]} ids")
        else:
            printm(f"## Segement visualization directory EXISTS for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
            
        
        

In [ ]:
session_video_file
